In [75]:
#Import
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
pd.options.mode.chained_assignment = None  # default='warn'
from wordcloud import WordCloud, STOPWORDS
from skimage import io
import json

In [186]:
# Load the data using pandas
dataDemo = pd.read_csv('m5_survey_data_demographics.csv')
dataTechno = pd.read_csv('m5_survey_data_technologies_normalised.csv')

dropdown_options = [
    {'label': 'Current Technology Usage', 'value': 'Current Technology Usage'},
    {'label': 'Future Technology Trend', 'value': 'Future Technology Trend'},
    {'label': 'Demographics', 'value': 'Demographics'}
]

# Initialize the Dash app
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Dashboard with multi pages",
            style={'textAlign': 'left', 'color': '#503D36', 'font-size': 24}),
    html.Div([
        html.Label("Select Dashboard:"),
        dcc.Dropdown(
            id='dropdown-dashboard',
            options=dropdown_options,
            value='Select a dashboard',
            placeholder='Dashboard',
            style={'width': '80%', 'padding': '3px', 'font-size': '14px'}
        )
    ]),
    html.Div([
        html.Div(id='output-container', className='chart-grid', style={'display': 'flex'}),
    ])
])

# Define the callback function to update the input container based on the selected statistics
@app.callback(
    Output(component_id='output-container', component_property='children'),
    [Input(component_id='dropdown-dashboard', component_property='value')]
)

def update_output_container(dropdown_dashboard):
    # Define a custom order for months
    month_order = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    if dropdown_dashboard == 'Current Technology Usage':
        # Capture Top 10 LanguageWorkedWith.
        languageWorkedWith = dataTechno.groupby('LanguageWorkedWith')['LanguageWorkedWith'].count().sort_values(ascending=False).head(10)
        languageWorkedWith = pd.DataFrame(languageWorkedWith).rename(columns = {"LanguageWorkedWith":"languageWorkedWith","LanguageWorkedWith":"count"}).reset_index()
        # Visualize as a Bar chart
        R_chart1 = dcc.Graph(figure=px.bar(languageWorkedWith, y='LanguageWorkedWith', x='count',
                                           color='count', 
                                           labels= {'count':'Number of surveys','LanguageWorkedWith':'Language'},
                                           #Include a proper Chart title
                                           title='Top 10 worked with Languages',
                                           #Include Show value labels feature
                                           text_auto=True))
        
        # Capture Top 10 DatabaseWorkedWith
        databaseWorkedWith = dataTechno.groupby('DatabaseWorkedWith')['DatabaseWorkedWith'].count().sort_values(ascending=False).head(10)
        databaseWorkedWith = pd.DataFrame(databaseWorkedWith).rename(columns = {"DatabaseWorkedWith":"databaseWorkedWith","DatabaseWorkedWith":"count"}).reset_index()
        # Visualize as a Column chart
        R_chart2 = dcc.Graph(figure=px.bar(databaseWorkedWith, x='DatabaseWorkedWith', y='count',
                                           color='count',
                                           labels= {'count':'Number of surveys','DatabaseWorkedWith':'Database'},
                                           #Include a proper Chart title
                                           title='Top 10 worked with Database',
                                           #Include Show value labels feature
                                           text_auto=True))

        # Capture PlatformWorkedWith
        platformWorkedWith = dataTechno.groupby('PlatformWorkedWith')['PlatformWorkedWith'].count()
        
        # Visualize as a Word cloud chart.
        wc = WordCloud(background_color='white', width=480, height=360,min_font_size=10).generate_from_frequencies(platformWorkedWith)
        wc.to_image().save('img.png',format='PNG')
        img = io.imread('img.png')
        R_chart3 = dcc.Graph(figure = px.imshow(img, title='Platform Worked With'))
        # Capture Top 10 WebFrameWorkedWith
        webFrameWorkedWith = dataTechno.groupby('WebFrameWorkedWith')['WebFrameWorkedWith'].count().sort_values(ascending=False).head(10)
        webFrameWorkedWith = pd.DataFrame(webFrameWorkedWith).rename(columns = {"WebFrameWorkedWith":"webFrameWorkedWith","WebFrameWorkedWith":"count"}).reset_index()
        # Visualize as a Hierarchy bubble chart
        R_chart4 = dcc.Graph(figure = px.scatter(webFrameWorkedWith,
                                                 x="WebFrameWorkedWith",
                                                 y="count",
                                                 size="count",
                                                 title="Top 10 WeFrame worked with",
                                                labels= {'count':'Number of surveys','WebFrameWorkedWith':'Web Frame'},
                                                 color="count"))
        return [
            html.Div(style={'display':'inline-block'},children=[
            html.Div(className='chart-item',style={'display':'flex'},
                     children=[html.Div(children=R_chart1,style={'display': 'inline-block','width':'50%'}), html.Div(children=R_chart2,style={'display': 'inline-block','width':'50%'})]),
            html.Br(),
            html.Div(className='chart-item',style={'display':'flex'},
                     children=[html.Div(children=R_chart3,style={'display': 'inline-block','width':'50%'}),html.Div(children=R_chart4,style={'display': 'inline-block','width':'50%'})])
        ])]
    elif dropdown_dashboard == 'Future Technology Trend':
        #Capture Top 10 LanguageDesireNextYear
        languageDesireNextYear = dataTechno.groupby('LanguageDesireNextYear')['LanguageDesireNextYear'].count().sort_values(ascending=False).head(10)
        languageDesireNextYear = pd.DataFrame(languageDesireNextYear).rename(columns = {"LanguageDesireNextYear":"languageWorkedWith","LanguageDesireNextYear":"count"}).reset_index()
        # Visualize as a Bar chart
        S_chart1 = dcc.Graph(figure=px.bar(languageDesireNextYear, y='LanguageDesireNextYear', x='count',
                                           color='count', 
                                           labels= {'count':'Number of surveys','LanguageDesireNextYear':'Language'},
                                           #Include a proper Chart title
                                           title='Top 10 desired Languages',
                                           #Include Show value labels feature
                                           text_auto=True))
        # Capture Top 10 DatabaseDesireNextYear.
        databaseDesireNextYear = dataTechno.groupby('DatabaseDesireNextYear')['DatabaseDesireNextYear'].count().sort_values(ascending=False).head(10)
        databaseDesireNextYear = pd.DataFrame(databaseDesireNextYear).rename(columns = {"DatabaseDesireNextYear":"databaseDesireNextYear","DatabaseDesireNextYear":"count"}).reset_index()
        # Visualize as a Column chart
        S_chart2 = dcc.Graph(figure=px.bar(databaseDesireNextYear, x='DatabaseDesireNextYear', y='count',
                                           color='count',
                                           labels= {'count':'Number of surveys','DatabaseDesireNextYear':'Database'},
                                           #Include a proper Chart title
                                           title='Top 10 desired Database',
                                           #Include Show value labels feature
                                           text_auto=True))
        
        # Capture PlatformDesireNextYear
        platformWorkedWith = pd.DataFrame(dataTechno.groupby('PlatformWorkedWith')['PlatformWorkedWith'].count())
        platformWorkedWith = platformWorkedWith.rename(columns = {'PlatformWorkedWith':'PlatformWorkedWith','PlatformWorkedWith':'count'})
        platformWorkedWith.reset_index(inplace=True)

        # Visualize as a Tree map chart
        S_chart3 = dcc.Graph(figure = px.treemap(platformWorkedWith,
                                                 values='count',
                                                 names='PlatformWorkedWith',
                                                 color='count',
                                                 path=['PlatformWorkedWith'],
                                                labels= {'count':'Number of surveys','PlatformWorkedWith':'Platform'},))

        # Capture Top 10 WebFrameDesireNextYear.
        webFrameDesireNextYear = dataTechno.groupby('WebFrameDesireNextYear')['WebFrameDesireNextYear'].count().sort_values(ascending=False).head(10)
        webFrameDesireNextYear = pd.DataFrame(webFrameDesireNextYear).rename(columns = {"WebFrameDesireNextYear":"WebFrameDesireNextYear","WebFrameDesireNextYear":"count"}).reset_index()
        # Visualize as a Hierarchy bubble chart
        S_chart4 = dcc.Graph(figure = px.scatter(webFrameDesireNextYear,
                                                 x="WebFrameDesireNextYear",
                                                 y="count",
                                                 size="count",
                                                 title="Top 10 WeFrame desired",
                                                labels= {'count':'Number of surveys','webFrameDesireNextYear':'Web Frame'},
                                                 color="count"))
        return [
            html.Div(style={'display':'inline-block'},children=[
            html.Div(className='chart-item',style={'display':'flex'},
                     children=[html.Div(children=S_chart1,style={'display': 'inline-block','width':'50%'}), html.Div(children=S_chart2,style={'display': 'inline-block','width':'50%'})]),
            html.Br(),
            html.Div(className='chart-item',style={'display':'flex'},
                     children=[html.Div(children=S_chart3,style={'display': 'inline-block','width':'50%'}),html.Div(children=S_chart4,style={'display': 'inline-block','width':'50%'})])
        ])]
    elif dropdown_dashboard == 'Demographics':
        # Use Filters for this tab feature to filter out entries of other types except Man and Woman from the data point Gender
        dataDemoFilter = dataDemo[(dataDemo['Gender']=='Man') | (dataDemo['Gender']=='Woman')]
        
        #Capture Respondent classified by Gender
        dfGender = dataDemoFilter.groupby('Gender')['Gender'].count().sort_values(ascending=False)
        dfGender = pd.DataFrame(dfGender).rename(columns={'Gender':'Gender','Gender':'count'}).reset_index()
        #Visualize as a Pie chart
        T_chart1 = dcc.Graph(figure = px.pie(dfGender,names='Gender',values='count',title='Respondents\' gender'))
        
        #Capture Respondent Count for Countries
        gapminder = pd.read_csv('count_respondent_per_country.csv') 
            #I counted the number of respondents per country, I extracted the ISO code of countries from https://geojson-maps.ash.ms/
            #Then I joined the data to link ISO code to count, I cleansed the data, found the missing codes... And I exported the data
            #to a csv (in github : https://github.com/Juliengm/ibm_captsone_project_202401/blob/b456a49a47b9383b944b16a28af05ec4a256883b/count_respondent_per_country.csv)
        #Fill the nan data with 0
        gapminder.fillna({'count':0})

        #Visualize as a map
        # create a map using choropleth
        T_chart2 = dcc.Graph(figure = px.choropleth(gapminder, locations='code', color='count', hover_name='country',color_continuous_scale='Oryel',
                            projection='natural earth', title = 'Location of respondents'))
        
        
        #Capture Respondent Count by Age
        countAge = dataDemoFilter.groupby('Age')['Age'].count()
        countAge = pd.DataFrame(countAge).rename(columns={'Age':'Age','Age':'count'}).reset_index().sort_values(by='Age')

        #Visualize as a Line chart
        T_chart3 = dcc.Graph(figure=px.line(countAge,
                               y='count',
                               x='Age',
                               title='Age repartition',
                               text='count').update_traces(textposition="bottom right"))
        
        #Capture Respondent Count by Gender, classified by Formal Education Level
        eduOrder = ['I never completed any formal education','Primary/elementary school','Professional degree (JD, MD, etc.)',
                            'Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)',
                            'Some college/university study without earning a degree','Associate degree','Bachelor’s degree (BA, BS, B.Eng., etc.)',
                            'Master’s degree (MA, MS, M.Eng., MBA, etc.)', 'Other doctoral degree (Ph.D, Ed.D., etc.)']
        ageGenderEdu = dataDemoFilter.groupby(['Gender','EdLevel'])[['Gender','EdLevel']].count()
        ageGenderEdu.drop('EdLevel',inplace=True,axis=1)
        ageGenderEdu = ageGenderEdu.rename(columns={'Gender':'count'}).reset_index()
        ageGenderEdu['EdLevel'] = pd.Categorical(ageGenderEdu['EdLevel'],categories = eduOrder, ordered = True)
        ageGenderEdu = ageGenderEdu.sort_values(by=['Gender','EdLevel'])
        
        #Visualize as a Stacked bar chart
        T_chart4 = dcc.Graph(figure=px.bar(ageGenderEdu, y='Gender', x='count',
                                           color='EdLevel',
                                           labels= {'count':'Number of surveys','Gender':'Gender'},
                                           #Include a proper Chart title
                                           title='Formal education level per gender',
                                           #Include Show value labels feature
                                           text_auto=True).update_layout(legend_x=0, legend_y=-1,xaxis_title=None))
        return [
            html.Div(style={'display':'inline-block'},children=[
            html.Div(className='chart-item',style={'display':'flex'},
                     children=[html.Div(children=T_chart1,style={'display': 'inline-block','width':'50%'}), html.Div(children=T_chart2,style={'display': 'inline-block','width':'50%'})]),
            html.Br(),
            html.Div(className='chart-item',style={'display':'flex'},
                     children=[html.Div(children=T_chart3,style={'display': 'inline-block','width':'50%'}),html.Div(children=T_chart4,style={'display': 'inline-block','width':'50%'})])
        ])]

In [187]:
if __name__ == '__main__':
    app.run_server(jupyter_mode="tab")


Dash app running on http://127.0.0.1:8050/


<IPython.core.display.Javascript object>